### Missing Data

Often data sources are incomplete, which means we will have missing data, we have 3 basic options for filling in missing data (we will personally have to make the decision for what is the right approach:

* Just keep the missing data points.
* Drop them missing data points (including the entire row)
* Fill them in with some other value.

Let's cover examples of each of these methods!

In [2]:
import findspark

In [3]:
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7/')

In [4]:
import pyspark

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('miss').getOrCreate()

In [7]:
df = spark.read.csv('ContainsNull.csv',header=True,inferSchema=True)

In [8]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Drop the missing data
We can use the .na functions for missing data. The drop command has the following parameters:

df.na.drop(how='any', thresh=None, subset=None)

* param how: 'any' or 'all'.

    If 'any', drop a row if it contains any nulls.
    If 'all', drop a row only if all its values are null.

* param thresh: int, default None

    If specified, drop rows that have less than `thresh` non-null values.
    This overwrites the `how` parameter.

* param subset: 
    optional list of column names to consider

In [9]:
df.na.drop().show()
# This removes all the rows where any column is null.

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [13]:
## We can also set the threshold value, to make dropping better.
## For example if thershold = 2, then na will drop out rows if there 2 nulls columns.

df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [14]:
## Instead of threshold, there is another parameter 'how'
## By defualt how = any means if any column value in a row is null it drops that row.
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [15]:
## set how to all
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [16]:
## with subset you can specify the column where you look for null to drop
df.na.drop(subset=["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [17]:
## Lets see how we can fill the nulls, na allso have fill()
df.na.fill('FILL VALUE').show()
## Below you see we fill the string type column containing null with 'FILL VALUE' as we pass the string

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|FILL VALUE| null|
|emp3|FILL VALUE|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [20]:
df.na.fill(0).show()
## see the behaviour with integers

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [19]:
df.schema

StructType(List(StructField(Id,StringType,true),StructField(Name,StringType,true),StructField(Sales,DoubleType,true)))

In [21]:
df.na.fill('NO NAME',subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|NO NAME| null|
|emp3|NO NAME|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



A very common practice is to fill values with the mean value for the column, for example:

In [23]:
from pyspark.sql.functions import mean

In [26]:
mean_value = df.select(mean(df['Sales'])).collect()

In [27]:
mean_sales = mean_value[0][0]

In [28]:
df.na.fill(mean_sales,['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [29]:
#one liner
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

